In [18]:
import os
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'autogluon'

In [ ]:
#!pip install autogluon

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
def read_data(path=None, data_name=None):
    folder_name = 'autogluon'
 
    if path is None:
        return None, None
    
    df = pd.read_csv(path)
    
    if data_name is None:
        basename = os.path.basename(path)
        data_name = basename[:len(basename) - 4]
    elif data_name.__contains__('.csv'):
        data_name = data_name[:len(data_name) - 4]
        
    X_train, X_test = train_test_split(df, test_size=0.25)
    
    if not os.path.exists(f'data/{folder_name}'):
        os.makedirs(f'data/{folder_name}')

    X_train_path = f'data/{folder_name}/{data_name}_train.csv'
    X_test_path = f'data/{folder_name}/{data_name}_test.csv'
    
    X_train.to_csv(X_train_path)
    X_test.to_csv(X_test_path)    
    
    return X_train_path, X_test_path

In [ ]:
data_name = 'bank-marketing.csv'
path = 'bank-marketing.csv'
y = 'Class'

X_train_path, X_test_path = read_data(path=path, data_name=data_name)

In [ ]:
train_data = TabularDataset(X_train_path)
test_data = TabularDataset(X_test_path)

In [ ]:
predictor = TabularPredictor(label=y, eval_metric='roc_auc').fit(train_data, time_limit=1560)

In [ ]:
test_acc = predictor.evaluate(test_data)

In [ ]:
predictor.leaderboard(test_data, silent=True,extra_metrics=
                      ['accuracy', 'balanced_accuracy', 'log_loss','mcc', 'recall_weighted', 'precision_weighted'])

In [ ]:
predictor.get_model_best()

In [ ]:
data_name = 'Moneyball.csv'
path = 'data/Moneyball.csv'
y = 'RS'

X_train_path, X_test_path = read_data(path=path, data_name=data_name)

In [ ]:
X_train_path, X_test_path

In [ ]:
train_data = TabularDataset(X_train_path)
test_data = TabularDataset(X_test_path)

In [ ]:
predictor = TabularPredictor(label=y, eval_metric='r2').fit(train_data, time_limit=3600)

In [ ]:
test_acc = predictor.evaluate(test_data)

In [ ]:
predictor.get_model_best()

In [ ]:
predictor.leaderboard()

In [26]:
import os
import numpy as np
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split

In [27]:
#!pip install h2o

In [32]:
h2o.init(max_mem_size="4G", nthreads=2)

In [ ]:
def read_data(path=None, data_name=None):
    folder_name = 'h2o'
 
    if path is None:
        return None, None
    
    df = pd.read_csv(path)
    
    if data_name is None:
        basename = os.path.basename(path)
        data_name = basename[:len(basename) - 4]
    elif data_name.__contains__('.csv'):
        data_name = data_name[:len(data_name) - 4]
        
    X_train, X_test = train_test_split(df, test_size=0.25)
    
    if not os.path.exists(f'data/{folder_name}'):
        os.makedirs(f'data/{folder_name}')

    X_train_path = f'data/{folder_name}/{data_name}_train.csv'
    X_test_path = f'data/{folder_name}/{data_name}_test.csv'
    
    X_train.to_csv(X_train_path)
    X_test.to_csv(X_test_path)    
    
    return X_train_path, X_test_path

In [ ]:
data_name = 'Moneyball.csv'
path = 'Moneyball.csv'
y = 'RS'
stopping_metric = 'AUTO'

X_train_path, X_test_path = read_data(path=path, data_name=data_name)

X_train = h2o.import_file(X_train_path)
X_test = h2o.import_file(X_test_path)

x = X_train.columns
x.remove(y)

In [ ]:
# Run AutoML for 1 hours
h2o_moneyball = H2OAutoML(max_runtime_secs=3600, keep_cross_validation_predictions=True, 
                          stopping_metric='RMSE', stopping_rounds=3)
h2o_moneyball.train(x=x, y=y, training_frame=X_train)

In [ ]:
h2o_moneyball.leader.model_performance(X_test)

In [ ]:
data_name = 'car.csv'
path = 'car.csv'
y = 'class'
stopping_metric = 'AUCPR'

X_train_path, X_test_path = read_data(path=path, data_name=data_name)

X_train = h2o.import_file(X_train_path)
X_test = h2o.import_file(X_test_path)

x = X_train.columns
x.remove(y)

In [ ]:
# Run AutoML for 1 hours
h2o_car = H2OAutoML(max_runtime_secs=1800, keep_cross_validation_predictions=True, 
                            stopping_metric=stopping_metric, stopping_rounds=3)
h2o_car.train(x=x, y=y, training_frame=X_train)

In [ ]:
h2o_car.leader.model_performance(X_test)